# pandas Integration

`bizdays` has a very handy integration with pandas `DataFrame`.

In [1]:
from bizdays import set_option, Calendar

set_option('mode', 'pandas')

When the option `mode` is set to `pandas` the calendar methods will return

- `Timestamp` instead of `datetime.date`
- `DatetimeIndex` instead of lists of dates
- `numpy.ndarray` instead of lists of integers (`bizdays`) and bools (`isbizday`)

In [2]:
cal = Calendar.load(filename='ANBIMA.cal')
cal

Calendar: ANBIMA
Start: 2000-01-01
End: 2078-12-25
Weekdays: Saturday, Sunday
Holidays: 948
Financial: True

In [3]:
cal.isbizday(['2014-01-12', '2014-01-13'])

array([False,  True])

In [4]:
cal.bizdays('2014-01-13', ['2015-01-13', '2016-01-13', None])

array([253, 503, <NA>], dtype=object)

In [5]:
cal.following('2015-12-25')

Timestamp('2015-12-28 00:00:00')

In [6]:
cal.following(['2015-12-28', '2015-12-25'])

DatetimeIndex(['2015-12-28', '2015-12-28'], dtype='datetime64[ns]', freq=None)

In [7]:
cal.seq('2014-01-02', '2014-01-07')

DatetimeIndex(['2014-01-02', '2014-01-03', '2014-01-06', '2014-01-07'], dtype='datetime64[ns]', freq=None)

In [8]:
cal.offset('2014-01-02', range(10))

DatetimeIndex(['2014-01-02', '2014-01-03', '2014-01-06', '2014-01-07',
               '2014-01-08', '2014-01-09', '2014-01-10', '2014-01-13',
               '2014-01-14', '2014-01-15'],
              dtype='datetime64[ns]', freq=None)

In [9]:
cal.getdate('15th bizday', 2002, 5)

Timestamp('2002-05-22 00:00:00')

In [10]:
cal.getdate('15th day', 2002, range(1, 13))

DatetimeIndex(['2002-01-15', '2002-02-15', '2002-03-15', '2002-04-15',
               '2002-05-15', '2002-06-15', '2002-07-15', '2002-08-15',
               '2002-09-15', '2002-10-15', '2002-11-15', '2002-12-15'],
              dtype='datetime64[ns]', freq=None)

In [11]:
cal.getbizdays(2021, range(1, 13))

array([20, 18, 23, 20, 21, 21, 22, 22, 21, 20, 20, 23])

## Example: series of payments

Let's create a series of payment dates and compute the business days between each payment date.

The payments are scheduled to occur in the 15th days of each month of 2021.

Start creating a DataFrame with year and month columns.

In [12]:
import pandas as pd

df = pd.DataFrame({
    'year': 2021,
    'month': range(1,13)
})
df

,year,month
0,2021,1
1,2021,2
2,2021,3
3,2021,4
4,2021,5
5,2021,6
6,2021,7
7,2021,8
8,2021,9
9,2021,10


Use `getdate` to get the payment dates.

In [13]:
df['payment_dates'] = cal.getdate('15th day', df['year'], df['month'])
df

,year,month,payment_dates
0,2021,1,2021-01-15
1,2021,2,2021-02-15
2,2021,3,2021-03-15
3,2021,4,2021-04-15
4,2021,5,2021-05-15
5,2021,6,2021-06-15
6,2021,7,2021-07-15
7,2021,8,2021-08-15
8,2021,9,2021-09-15
9,2021,10,2021-10-15


The payments happen in business days so the dates must be shifted to the next working date.

In [14]:
df['fixing'] = cal.following(df['payment_dates'])
df

,year,month,payment_dates,fixing
0,2021,1,2021-01-15,2021-01-15
1,2021,2,2021-02-15,2021-02-17
2,2021,3,2021-03-15,2021-03-15
3,2021,4,2021-04-15,2021-04-15
4,2021,5,2021-05-15,2021-05-17
5,2021,6,2021-06-15,2021-06-15
6,2021,7,2021-07-15,2021-07-15
7,2021,8,2021-08-15,2021-08-16
8,2021,9,2021-09-15,2021-09-15
9,2021,10,2021-10-15,2021-10-15


`bizdays` compute the amount of business days between each payment.

In [15]:
df['business_days'] = cal.bizdays(df['fixing'].shift(), df['fixing'])
df

,year,month,payment_dates,fixing,business_days
0,2021,1,2021-01-15,2021-01-15,<NA>
1,2021,2,2021-02-15,2021-02-17,21
2,2021,3,2021-03-15,2021-03-15,18
3,2021,4,2021-04-15,2021-04-15,22
4,2021,5,2021-05-15,2021-05-17,21
5,2021,6,2021-06-15,2021-06-15,20
6,2021,7,2021-07-15,2021-07-15,22
7,2021,8,2021-08-15,2021-08-16,22
8,2021,9,2021-09-15,2021-09-15,21
9,2021,10,2021-10-15,2021-10-15,21
